<a href="https://colab.research.google.com/github/mickeykubo/analytics_book/blob/master/linear_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python言語によるビジネスアナリティクス

## 実務家のための最適化，統計分析，機械学習（近代科学社）
 
## PuLPとGurobi/Pythonによる線形最適化問題のモデリング


* 線形最適化ラッパー mypulp
* 簡単な例題と双対最適解
* 輸送問題，多品種輸送問題
* 栄養問題

In [1]:
!pip install mypulp

    100% |████████████████████████████████| 13.6MB 2.5MB/s 
  Running setup.py bdist_wheel for mypulp ... - done
  Stored in directory: /root/.cache/pip/wheels/f3/57/90/17b22caa69d04beb2965bfbb8133c57b58800feae034858822
  Running setup.py bdist_wheel for pulp ... - \ | / - \ | done
  Stored in directory: /root/.cache/pip/wheels/3b/f1/f5/f6d0aac56dd154c2593d3fa5e901aa9bc32a51bd8594b4bf8e
Successfully built mypulp pulp


### 線形最適化ソルバー(モデラー）
-----------


* Gurobi（商用，アカデミックフリー）のソルバー
  * 独自のPythonインターフェイス（「あたらしい数理最適化」（近代科学社）で採用）
 * 凸2次（制約）整数，2次錐最適化
* PuLP （MITライセンス）のモデラー
  * メインソルバーはCBC(COIN Branch & Cut; EPLライセンス），その他様々なソルバーと連携(MPSフォーマット経由）
  * Gurobiと同じインターフェイスをもつ**mypulp**モジュールを使う




## 例題　

* トンコケ，コケトン，ミックスの丼を販売
* 資源制約の下で，利益を最大化


変数

* トンコケ丼 $x_1$，コケトン丼 $x_2$，ミックス丼 $x_3$

* 定式化

\begin{array}{l c c c c c}
 \mbox{maximize} & 15 x_1  & + 18 x_2 & +30 x_3 &      & \\
 \mbox{subject to}   & 2x_1   & + x_2   & + x_3   & \leq & 60 \\
               &  x_1   & + 2 x_2 & + x_3  &\leq  & 60 \\
               &        &         &   x_3  &\leq  & 30 \\
               &        &         & x_1,x_2,x_3 & \geq & 0 
 \end{array}

In [2]:
#from gurobipy import *
from mypulp import *
model = Model('lo1')

x1 = model.addVar(name='x1')
x2 = model.addVar(name='x2')
x3 = model.addVar(ub=30., name='x3')

model.update() #Gurobiの怠惰な更新(lazy update)という仕様（忘れずに！）
    
model.addConstr(2*x1 + x2 + x3 <= 60)
# 別の定義方法 1
#L1 = LinExpr([2,1,1],[x1,x2,x3]) #線形表現(式）
# 別の定義方法 2
#L1 = LinExpr()     #線形表現(式）
#L1.addTerms(2,x1)  #項を追加
#L1.addTerms(1,x2)
#L1.addTerms(1,x3)
#model.addConstr(lhs=L1,sense='<',rhs=60)  #制約を追加

model.addConstr(x1 + 2*x2 + x3 <= 60)

model.setObjective(15*x1 + 18*x2 + 30*x3, GRB.MAXIMIZE)

model.optimize()

if model.Status == GRB.Status.OPTIMAL:
    print('Opt. Value=',model.ObjVal)
    for v in model.getVars():
        print(v.VarName,v.X)

Opt. Value= 1230.0
x1 10.0
x2 10.0
x3 30.0


### モデルファイルの出力
------------
* model.write('ファイル名.lp')で **LPフォーマット**(Linear Programming (LP) format)で保存

* model.write('ファイル名.mps')で**MPSフォーマット** (Mathematical Programming System (MPS) format) で保存
  * 可読性はないが，ほとんどの最適化ソルバーが対応している古典的な書式
  
* 注意：Gurobiの場合にはmodel.update()を直前にするのを忘れずに

* PuLPだと print(model) でも画面にLPフォーマットを出力


In [7]:
model.write('lo1.lp')
model.write('lo1.mps')
print('MPS file =========================')
!cat lo1.lp

lo1:
MAXIMIZE
15*x1 + 18*x2 + 30*x3 + 0
SUBJECT TO
c_1: 2 x1 + x2 + x3 <= 60

c_2: x1 + 2 x2 + x3 <= 60

VARIABLES
x1 <= 1e+100 Continuous
x2 <= 1e+100 Continuous
x3 <= 30 Continuous

MPS file =========================
\* lo1 *\
Maximize
OBJ: 15 x1 + 18 x2 + 30 x3
Subject To
c_1: 2 x1 + x2 + x3 <= 60
c_2: x1 + 2 x2 + x3 <= 60
Bounds
x1 <= 1e+100
x2 <= 1e+100
x3 <= 30
End


## 双対問題
-----------

### 資源（豚肉，鶏肉，牛肉）100グラムの価値を推定

主問題
------------

\begin{array}{l c c c c c}
 \mbox{maximize} & 15 x_1  & + 18 x_2 & +30 x_3 &      & \\
 \mbox{subject to}   & 2x_1   & + x_2   & + x_3   & \leq & 60 \\
               &  x_1   & + 2 x_2 & + x_3  &\leq  & 60 \\
               &        &         &   x_3  &\leq  & 30 \\
               &        &         &x_1,x_2,x_3  & \geq & 0 
 \end{array}



双対問題
--------------

\begin{array}{l c c c c c}
 \mbox{minimize} & 60 \pi_1 & + 60 \pi_2& +30 \pi_3 &      & \\
 \mbox{subject to}   & 2\pi_1   & + \pi_2   &         & \geq & 15 \\
               &  \pi_1   & + 2\pi_2  &         &\geq  & 18 \\
               &  \pi_1   & +\pi_2    &  +\pi_3   &\geq  & 30 \\
               &          &           & \pi_1,\pi_2,\pi_3  & \geq & 0 
 \end{array}
 
最適双対変数は 4, 7, 19 

豚肉は百グラム$400$ 円，鶏肉は百グラム $700$ 円， 牛肉は百グラム
$1900$ 円の価値をもつ

In [8]:
for c in model.getConstrs():
    print( c.ConstrName, c.Pi )

c_1 4.0
c_2 7.0


## 輸送問題
---------------

**quicksum**と**multidict**を用いた一般的な記述法

|顧客 $i$ | 1  |  2  | 3  | 4 | 5 | 
|--:|----:|----:|---:|---:|---:|
|需要量 $d_i$  |80 | 270 | 250 | 160 | 180 |   
|工場 $j$|  輸送費用 | $c_{ij}$ |   |  |   | 容量 $M_j$  |
|1      | 4 | 5 | 6 | 8 | 10 |  500 |
|2      | 6  |4 | 3 | 5 | 8 |  500 |
|3      | 9  | 7 | 4 | 3 | 4 |  500 |




$x_{ij}= \mbox{工場 $j$ から顧客 $i$ に輸送される量} $

定式化

\begin{array}{l l l} 
\nonumber
 \mbox{ minimize } & \displaystyle\sum_{i \in I} \displaystyle\sum_{j \in J} c_{ij} x_{ij}  &     \\
 \mbox{ subject to   } &
\nonumber
 \displaystyle\sum_{j \in J} x_{ij} =d_i &  \forall  i \in I  \\
\nonumber
   & \displaystyle\sum_{i \in I} x_{ij} \leq M_j &  \forall  j \in J \\   
\nonumber
                 & x_{ij} \geq 0 & \forall  i \in I; j \in J  
\end{array}

In [9]:
# multidict の使用法
from mypulp import *
name, height, weight=multidict({'Taro':[145,30],'Hanako':[138,34],'Simon':[150,45]})
print(name)
print(height)
print(weight)

['Taro', 'Hanako', 'Simon']
{'Taro': 145, 'Hanako': 138, 'Simon': 150}
{'Taro': 30, 'Hanako': 34, 'Simon': 45}


In [10]:
# quicksum の使用法
from mypulp import *
model=Model()
a=[5,4,2]
x=[model.addVar() for i in range(3)]
L = quicksum(a[i]*x[i] for i in range(3))
print(L)

5*x_1 + 4*x_2 + 2*x_3


In [11]:
#from gurobipy import *
from mypulp import *
I,d = multidict({1:80, 2:270, 3:250 , 4:160, 5:180}) # demand
J,M = multidict({1:500, 2:500, 3:500})               # capacity
c = {(1,1):4,    (1,2):6,    (1,3):9,  # cost
     (2,1):5,    (2,2):4,    (2,3):7,
     (3,1):6,    (3,2):3,    (3,3):4,
     (4,1):8,    (4,2):5,    (4,3):3,
     (5,1):10,   (5,2):8,    (5,3):4,
     }

model = Model('transportation')
x = {}
for i in I:
    for j in J:
        x[i,j] = model.addVar(vtype='C', name='x({0},{1})'.format(i,j))
model.update()

for i in I:
    model.addConstr(quicksum(x[i,j] for j in J if (i,j) in x) == d[i],
                    name='Demand({0})'.format(i))
for j in J:
    model.addConstr(quicksum(x[i,j] for i in I if (i,j) in x) <= M[j], 
                    name='Capacity({0})'.format(j))
model.setObjective(quicksum(c[i,j]*x[i,j]  for (i,j) in x), GRB.MINIMIZE)

model.optimize()
print( 'Optimal value:', model.ObjVal)

EPS = 1.e-6
for (i,j) in x:
    if x[i,j].X > EPS:
        print('{0:>5} from factory {1:>2} to customer {2:>2}'.format(x[i,j].X,j,i) )
        
print ('{0:>15}: {1:>8} , {2:>4}'.format('Const. Name', 'Slack', 'Dual'))
for c in model.getConstrs():
    print ('{0:>15}: {1:>8} , {2:>4}'.format(c.ConstrName,c.Slack,c.Pi))

Optimal value: 3370.0
 80.0 from factory  1 to customer  1
270.0 from factory  2 to customer  2
230.0 from factory  2 to customer  3
 20.0 from factory  3 to customer  3
160.0 from factory  3 to customer  4
180.0 from factory  3 to customer  5
    Const. Name:    Slack , Dual
      Demand(1):     -0.0 ,  4.0
      Demand(2):     -0.0 ,  5.0
      Demand(3):     -0.0 ,  4.0
      Demand(4):     -0.0 ,  3.0
      Demand(5):     -0.0 ,  4.0
    Capacity(1):    420.0 ,  0.0
    Capacity(2):     -0.0 , -1.0
    Capacity(3):    140.0 ,  0.0


## 多品種輸送問題
---------------
 疎な問題の扱い方と ** tuplelist **

$
  x_{ijk}= \mbox{工場 $j$ から顧客 $i$ に製品 $k$ が輸送される量}
$

 工場1では製品2,4を，工場2では製品1,2,3を，工場3では製品2,3,4を製造可能

`produce = {1:[2,4], 2:[1,2,3], 3:[2,3,4]}`  


定式化

\begin{array}{l l l} 
\nonumber
 \mbox{ minimize } & \displaystyle\sum_{i \in I} \displaystyle\sum_{j \in J} 
 \displaystyle\sum_{k \in K} c_{ijk} x_{ijk}  &     \\
 \mbox{ subject to   } &
\nonumber
 \displaystyle\sum_{j \in J} x_{ijk} =d_{ik} &  \forall  i \in I; k \in K  \\
\nonumber
   & \displaystyle\sum_{i \in I} \displaystyle\sum_{k \in K} x_{ijk} \leq M_j &  \forall  j \in J  \\   
\nonumber
                 & x_{ijk} \geq 0 & \forall  i \in I; j \in J; k \in K  
\end{array}

In [0]:
# tuplelist の使用法
from mypulp import *
T = tuplelist([('Sara','Apple'),('Taro','Pear'),('Jiro','Orange'),('Simon','Apple')])
print( T.select('*','Apple') )               

[('Sara', 'Apple'), ('Simon', 'Apple')]


In [12]:
from mypulp import *
d = {(1,1):80,   (1,2):85,   (1,3):300,  (1,4):6,
     (2,1):270,  (2,2):160,  (2,3):400,  (2,4):7,
     (3,1):250,  (3,2):130,  (3,3):350,  (3,4):4,
     (4,1):160,  (4,2):60,   (4,3):200,  (4,4):3,
     (5,1):180,  (5,2):40,   (5,3):150,  (5,4):5
     }

I = set([i for (i,k) in d])
K = set([k for (i,k) in d])
J,M = multidict({1:3000, 2:3000, 3:3000})  # capacity
produce = {1:[2,4], 2:[1,2,3], 3:[2,3,4]}  # products that can be produced in each facility
weight = {1:5, 2:2, 3:3, 4:4}              # {commodity: weight<float>}

cost = {(1,1):4,  (1,2):6, (1,3):9,        # {(customer,factory): cost<float>}
        (2,1):5,  (2,2):4, (2,3):7,
        (3,1):6,  (3,2):3, (3,3):4,
        (4,1):8,  (4,2):5, (4,3):3,
        (5,1):10, (5,2):8, (5,3):4
        }

c = {}
for i in I:
    for j in J:
        for k in produce[j]:
            c[i,j,k] = cost[i,j] * weight[k]            
model = Model('multi-commodity transportation')

x = {}
for (i,j,k) in c:
    x[i,j,k] = model.addVar(vtype='C', name='x({0},{1},{2})'.format(i,j,k))
model.update()
arcs = tuplelist([(i,j,k) for (i,j,k) in x])
for i in I:
    for k in K:
        model.addConstr(quicksum(x[i,j,k] for (i,j,k) in arcs.select(i,'*',k)) == d[i,k],
                        'Demand({0},{1})'.format(i,k))
for j in J:
    model.addConstr(quicksum(x[i,j,k] for (i,j,k) in arcs.select('*',j,'*')) <= M[j],
                    'Capacity({0})'.format(j))

model.setObjective(quicksum(c[i,j,k]*x[i,j,k]  for (i,j,k) in x), GRB.MINIMIZE)
model.optimize()
print ('Optimal value:',model.ObjVal)

EPS = 1.e-6
for (i,j,k) in x:
    if x[i,j,k].X > EPS:
        print ('{0:>5}s units {1:>2} from {2:>2} to {3:>2}'.format(x[i,j,k].X,k,j,i))

Optimal value: 43536.0
 85.0s units  2 from  1 to  1
  6.0s units  4 from  1 to  1
 80.0s units  1 from  2 to  1
300.0s units  3 from  2 to  1
  7.0s units  4 from  1 to  2
270.0s units  1 from  2 to  2
160.0s units  2 from  2 to  2
400.0s units  3 from  2 to  2
250.0s units  1 from  2 to  3
130.0s units  2 from  2 to  3
350.0s units  3 from  2 to  3
  4.0s units  4 from  3 to  3
160.0s units  1 from  2 to  4
 60.0s units  2 from  3 to  4
200.0s units  3 from  3 to  4
  3.0s units  4 from  3 to  4
180.0s units  1 from  2 to  5
 40.0s units  2 from  3 to  5
150.0s units  3 from  3 to  5
  5.0s units  4 from  3 to  5


### 栄養問題（実行不可能性の取り扱い）
----------------

|栄養素 $N$ | Cal | Carbo | Protein | VitA | VitC | Calc | Iron |価格　|
|--:|----:|----:|---:|---:|---:|---:|---:|---:|
|商品名 $F$ | |  | $n_{ij}$  |  |  | |  |$c_j$　
| CQPounder | 556| 39| 30| 147| 10| 221| 2.4| 360|
|Big M | 556| 46| 26| 97 | 9 | 142| 2.4| 320|
|FFilet | 356| 42| 14| 28 | 1 | 76 | 0.7| 270|
|Chicken | 431| 45| 20| 9 | 2 | 37 | 0.9| 290|
|Fries| 249| 30| 3 | 0 | 5 | 7 | 0.6| 190|
|Milk | 138| 10| 7 | 80 | 2 | 227| 0 | 170|
|VegJuice | 69 | 17| 1 | 750| 2 | 18 | 0 | 100|
|上限 $a_i$ | 3000| 375| 60| 750| 100| 900| 7.5|
|下限 $b_i$ | 2000| 300| 50| 500| 85| 660| 6.0|




定式化 

$x_j$ は商品 $j$ の購入量(実数）

\begin{array}{l l l}
   \mbox{minimize}    & \displaystyle\sum_{j \in F} c_j x_j          &  \\
   \mbox{subject to}  & a_i \leq \displaystyle\sum_{j \in F} n_{ij} x_j \leq b_i &  i \in N    \\
                      & x_j \geq 0  &     j \in F
\end{array}

$d_i$: 不足変数

$s_i$: 超過変数



改良した制約

$$a_i - d_i \leq \displaystyle\sum_{j \in F} n_{ij} x_j \leq b_i +s_i  \ \   i \in N$$

変更した目的関数（Mは大きな数）

\begin{array}{l l l}
   \mbox{minimize}    & \displaystyle\sum_{j \in F} c_j x_j  + \displaystyle\sum_{i \in N} M (d_i+s_i)  &  
\end{array}

In [13]:
F, c, n = multidict({      
    'CQPounder':  [ 360, {'Cal':556, 'Carbo':39, 'Protein':30, 'VitA':147,
                          'VitC': 10, 'Calc':221, 'Iron':2.4}], 
    'Big M'    :  [ 320, {'Cal':556, 'Carbo':46, 'Protein':26, 'VitA':97,
                          'VitC':  9, 'Calc':142, 'Iron':2.4}], 
    'FFilet'   :  [ 270, {'Cal':356, 'Carbo':42, 'Protein':14, 'VitA':28, 
                          'VitC':  1, 'Calc': 76, 'Iron':0.7}], 
    'Chicken'  :  [ 290, {'Cal':431, 'Carbo':45, 'Protein':20, 'VitA': 9, 
                          'VitC':  2, 'Calc': 37, 'Iron':0.9}], 
    'Fries'    :  [ 190, {'Cal':249, 'Carbo':30, 'Protein': 3, 'VitA': 0, 
                          'VitC':  5, 'Calc':  7, 'Iron':0.6}], 
    'Milk'     :  [ 170, {'Cal':138, 'Carbo':10, 'Protein': 7, 'VitA':80,
                          'VitC':  2, 'Calc':227, 'Iron': 0}], 
    'VegJuice' :  [ 100, {'Cal': 69, 'Carbo':17, 'Protein': 1, 'VitA':750,
                          'VitC':  2, 'Calc':18,  'Iron': 0}] })
N, a, b = multidict({       
    'Cal'     : [ 2000,  3000],
    'Carbo'   : [  300,  375 ],
    'Protein' : [   50,   60 ],
    'VitA'    : [  500,  750 ],
    'VitC'    : [   85,  100 ],
    'Calc'    : [  660,  900 ],
    'Iron'    : [  6.0,  7.5 ]})
model = Model('modern diet')
x,s,d = {},{},{}
for j in F:
    x[j] = model.addVar(vtype='C', name='x({0})'.format(j))
for i in N:
    s[i] = model.addVar(vtype='C', name='surplus({0})'.format(i))
    d[i] = model.addVar(vtype='C', name='deficit({0})'.format(i))
model.update()
for i in N:
    model.addConstr(quicksum(n[j][i]*x[j] for j in F) >= a[i]-d[i],'NutrLB({0})'.format(i))
    model.addConstr(quicksum(n[j][i]*x[j] for j in F) <= b[i]+s[i],'NutrUB({0})'.format(i))
model.setObjective(quicksum(c[j]*x[j]  for j in F)+
                   quicksum(9999*d[i]+9999*s[i] for i in N), GRB.MINIMIZE )
model.optimize()
status = model.Status
if status == GRB.Status.OPTIMAL:
    print ("Optimal value:",model.ObjVal)
    for j in F:
        if x[j].X > 0:
            print (j,x[j].X)
    for i in N:
        if d[i].X > 0:
            print ('deficit of {0} ={1}'.format(i,d[i].X))
        if s[i].X > 0:
            print ('surplus of {0} ={1}'.format(i,s[i].X))

Optimal value: 265119.18554951996
CQPounder 0.013155307
Fries 10.422665
Milk 2.5154631
VegJuice 0.72910549
deficit of VitC =26.265987


fin.      